<a href="https://colab.research.google.com/github/dixitmudit/PACE/blob/main/examples/pace-results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mace-torch
!pip install ase
!pip install weas-widget
!pip install pymatgen

In [ ]:
import numpy as np
import pandas as pd
from ase import Atoms
from ase.io import read, write
import matplotlib.pyplot as plt
import torch
from mace.calculators import MACECalculator

In [ ]:
from urllib.request import urlopen
url = "https://raw.githubusercontent.com/dixitmudit/PACE/refs/heads/main/examples/data/Fe-Ni.vasp.VASP"
base_file = urlopen(url).read().decode("utf-8")

url = "https://raw.githubusercontent.com/dixitmudit/PACE/refs/heads/main/examples/data/Li2S.VASP"
adsorbate_file = urlopen(url).read().decode("utf-8")

In [ ]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

bridge = AseAtomsAdaptor()

try:
  base=read(filename=base_file)
  adsorbate=read(filename=adsorbate_file)
except:
  base = Structure.from_str(base_file, fmt="poscar")
  base = bridge.get_atoms(base) # convert Structure to Atoms

  adsorbate = Structure.from_str(adsorbate_file, fmt="poscar")
  adsorbate = bridge.get_atoms(adsorbate) # convert Structure to Atoms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Running on: ', device)
mace_calc = MACECalculator('/path/to/model/here', device=device)

In [ ]:
from pace import PACE
pace = PACE(base=base, adsorbate=adsorbate, division=100, z_levels=[1.25, 1.75, 2 ], arch='mace')

results = pace.screen(calculator=mace_calc, fig_save_at='figs', mlip_optimization=3,euler_angles=[0, -45, -90], make_countours=4)
optmised_structure = results['screened_structures'][0]

In [ ]:
from weas_widget import WeasWidget
viewer = WeasWidget()
viewer.from_ase(optmised_structure)
viewer